In [4]:
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [5]:
from datasets import *

In [6]:
(x,y)=DATA['iris']

In [7]:
def normalize_cols(m):
    col_max=m.max(axis=0)
    col_min=m.min(axis=0)
    return (m-col_min)/(col_max-col_min)

In [8]:
x=normalize_cols(x)

In [9]:
(n,m)=x.shape

In [10]:
import matplotlib
classes=np.unique(y)
colors=matplotlib.pyplot.cm.gist_rainbow(np.linspace(0,1,len(classes)))
#colors=matplotlib.pyplot.cm.rainbow(np.linspace(0,1,len(classes)))
cm = matplotlib.colors.ListedColormap(colors)
axiscolors=matplotlib.pyplot.cm.rainbow(np.linspace(0,1,m))

In [11]:
def StarCoordinates(m):
	theta=np.pi*2/m
	v=np.zeros((m,2))
	for i in range(m):
		v[i,0]=np.cos(i*theta)
		v[i,1]=np.sin(i*theta)
	return v

In [12]:
SC=StarCoordinates(m)

In [13]:
def Polyviz(x,anchor,pm):
	#alpha
	m=len(x)
	theta=np.zeros((m,2))
	
	for i in range(m):
		theta[i,0]=anchor[pm[i],0]*x[pm[i]]+anchor[pm[(i+1)%m],0]*(1.0-x[pm[i]])
		theta[i,1]=anchor[pm[i],1]*x[pm[i]]+anchor[pm[(i+1)%m],1]*(1.0-x[pm[i]])
		
	#anchor=np.array([np.cos(theta),np.sin(theta)])
	px=np.dot(x,theta)/np.sum(x)
	return px

In [14]:
def circleradviz(X,a):
	#p=sum 
	(n,m)=X.shape
	Y=np.zeros((n,2))
	alpha=a
	order=np.argsort(a)
	anchor=np.zeros((m,2))
	for i in range(m):
		anchor[i]=np.array([np.cos(alpha[i]),np.sin(alpha[i])])
	
	for i in range(n):
		Y[i]=Polyviz(X[i],anchor,order)
	return Y

In [15]:
def Classifier(x,y):
	clf=LDA()
	clf.fit(x,y)
	ac=clf.score(x,y)
	#ac=silhouette(x, y, metric='sqeuclidean')
	return ac

In [16]:
A1,A2,A3,A4=0.0,np.pi/2,np.pi,3*np.pi/2

In [19]:
def func_iradviz(A1,A2,A3,A4):
    Alpha=np.array([A1,A2,A3,A4])
    m=len(Alpha)
    anchor=np.zeros((m,2))
    for i in range(m):
        anchor[i,0]=np.cos(Alpha[i])
        anchor[i,1]=np.sin(Alpha[i])
        
    Y=circleradviz(x,Alpha)
    #draw Radviz
    plt.figure(figsize=(5,5))
    plt.scatter(Y[:,0],Y[:,1],c=y,cmap=cm,marker='.',s=10)
    #draw unit circle
    t = np.linspace(0,2*np.pi, 100)
    plt.plot(np.cos(t),np.sin(t),alpha=0.95)
    #draw alpha point
    (n,m)=x.shape
    al=np.array([A1,A2,A3,A4])
    
    pm=np.argsort(al)
    for i in range(m):
        t1=al[pm[i]]
        t2=al[pm[(i+1)%m]]
        plt.plot([np.cos(t1),np.cos(t2)],[np.sin(t1),np.sin(t2)],color=axiscolors[pm[i]])
        plt.scatter(np.cos(t1),np.sin(t1),color="red",alpha=0.5);
        plt.text(0.5*(np.cos(t1)+np.cos(t2)),0.5*(np.sin(t1)+np.sin(t2)),'A'+str(pm[i]+1),color="black")
    
    ac=Classifier(Y,y)
    plt.text(-1,1,str(ac*100)[:5]+'%')
    plt.xlim(-1.2,1.2)
    plt.ylim(-1.2,1.2)
    for i in range(m):
        #np.linspace(V[i,0],V[i,1], 30)
        #plt.scatter(anchor[i,0],anchor[i,1],color=axiscolors[i],size=al[i]*10,alpha=0.95)
        plt.scatter(anchor[i,0],anchor[i,1],color=axiscolors[i],alpha=0.95)
        #plt.text((1+0.1)*anchor[i,0],(1+0.1)*anchor[i,1],'A'+str(i+1),color="black")
    plt.show()
    return (A1,A2,A3,A4)

In [20]:
p=interactive(func_iradviz,A1=(0,2*np.pi,0.1),A2=(0,2*np.pi,0.1),A3=(0,2*np.pi,0.1),A4=(0,2*np.pi,0.1))
display(p)

interactive(children=(FloatSlider(value=3.1, description='A1', max=6.283185307179586), FloatSlider(value=3.1, description='A2', max=6.283185307179586), FloatSlider(value=3.1, description='A3', max=6.283185307179586), FloatSlider(value=3.1, description='A4', max=6.283185307179586), Output()), _dom_classes=('widget-interact',))